In [1]:
from __future__ import absolute_import, division, print_function
# Import TensorFlow >= 1.10 and enable eager execution
%tensorflow_version 1.x
import tensorflow as tf
tf.enable_eager_execution()
print(tf.__version__)

1.15.0


In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#path to settings file
#get input from user
dataset_folder = '/content/gdrive/My Drive/ChatBot_Dataset/twitter_30K/'
settings_file_path = dataset_folder + 'settings_qam3c.json'

In [4]:
#@title
import json
import unicodedata
import re
from datetime import datetime
from __future__ import absolute_import, division, print_function
from sklearn.model_selection import train_test_split
import numpy as np
import os
import time


class Settings():
    def __init__ (self,settings_file_path,parent_folder):
        self.settings = settings_file_path
        self.read_settings(parent_folder )

    def read_settings(self,parent_folder):
        with open(self.settings) as f:
            jsonsettings = json.load(f)
            
        self.model_path = parent_folder + jsonsettings["model_path"]
        self.qam3_path = parent_folder + jsonsettings["qam3_path"]
        self.qam5_path = parent_folder + jsonsettings["qam5_path"]
        self.coreref_path = parent_folder + jsonsettings["coreref_path"]
       
        self.training_set_path = parent_folder + jsonsettings["training_set_path"]
        self.validation_set_path = parent_folder + jsonsettings["validation_set_path"]
        self.test_set_path = parent_folder + jsonsettings["test_set_path"]
        self.training_size = jsonsettings["training_size"]
        self.validation_size = jsonsettings["validation_size"]
        
        self.embedding_type = jsonsettings["embedding_type"]
        self.encoder_type = jsonsettings["encoder_type"]
        self.rnn_type = jsonsettings["rnn_type"]
        self.internal_feedback = jsonsettings["internal_feedback"]
        self.dropout = jsonsettings["dropout"]
        self.recurrent_dropout = jsonsettings["recurrent_dropout"]
        
        self.checkpoint_path = self.model_path + '/' + jsonsettings["checkpoint_path"]
        self.checkpoint_path_qam3 = self.qam3_path + '/' + jsonsettings["checkpoint_path"]
        self.checkpoint_path_qam5 = self.qam5_path + '/' + jsonsettings["checkpoint_path"]
        self.checkpoint_path_coreref = self.coreref_path + '/' + jsonsettings["checkpoint_path"]

        self.embedding_dim = jsonsettings["embedding_dim"]
        self.units = jsonsettings["units"]
        self.epochs = jsonsettings["epochs"]
        self.batch_size = jsonsettings["batch_size"]
        self.beam_size = jsonsettings["beam_size"]

        self.input_vocab_path = parent_folder  + jsonsettings["input_vocab_path"]
        self.input_vocab_size = jsonsettings["input_vocab_size"]
        self.output_vocab_path = parent_folder  + jsonsettings["output_vocab_path"]
        self.output_vocab_size = jsonsettings["output_vocab_size"]
        self.label_vocab_path_qam3 = parent_folder + jsonsettings["label_vocab_path_qam3"]
        self.label_vocab_size_qam3 = jsonsettings["label_vocab_size_qam3"]
        self.label_vocab_path_qam5 = parent_folder + jsonsettings["label_vocab_path_qam5"]
        self.label_vocab_size_qam5 = jsonsettings["label_vocab_size_qam5"]
        self.label_vocab_path_coreref = parent_folder + jsonsettings["label_vocab_path_coreref"]
        self.label_vocab_size_coreref = jsonsettings["label_vocab_size_coreref"]

        self.input_max_len = jsonsettings["input_max_len"]
        self.output_max_len = jsonsettings["output_max_len"]
        self.label_max_len = jsonsettings["label_max_len"]
        
        self.training_loss_path = self.model_path + '/' + jsonsettings["training_loss_path"]
        self.validation_loss_path = self.model_path + '/' + jsonsettings["validation_loss_path"]
        self.predictions_path = self.model_path + '/' + jsonsettings["predictions_path"]

    def update_settings(self,jsonfilepath,key,value):
        jsonFile = open(jsonfilepath, "r") # Open the JSON file for reading
        data = json.load(jsonFile) # Read the JSON into the buffer
        jsonFile.close() # Close the JSON file
    
        data[key] = str(value)
    
        ## Save our changes to JSON file
        jsonFile = open(jsonfilepath, "w")
        jsonFile.write(json.dumps(data))
        jsonFile.close()
                
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def removelastquote(word):
    if len(word) > 1:
        if (( word[0]) == "'"):
           word = word [1:]
        if (( word[len(word)-1]) == "'"):
           word = word [0:-1]
        return word
    else:
        return word
      
def clean_string(line):
    line = line.lower()
    #line = change_quote(line)
    #line = line.replace("'","")
    #line = line.replace("`","'")
    line = re.sub(r"[^a-z0123456789'!?,.-]+", " ", line)
    line = re.sub(r'["?"]+', " ? ", line)
    line = re.sub(r'["."]+', " . ", line)
    line = re.sub(r'[","]+', " , ", line)
    line = re.sub(r'["!"]+', " ! ", line)
    line = re.sub(r'[" "]+', " ", line)
    return line.strip()

def preprocess_sentence_word(sentence):
    sentence = clean_string(sentence)  
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

def preprocess_user_input_word(sentence,input_vocab):
    sentence = clean_string(sentence)  
    #to do: may be try adding a <unk> token instead
    new_sentence=""
    for word in sentence.split(' '):
        if word in input_vocab.idx2word.values():
            new_sentence = new_sentence + word + ' '
        else:
            new_sentence = new_sentence + '<unk>' + ' '
    #new_sentence = re.sub(r'[" "]+', " ", new_sentence)
    new_sentence = new_sentence.strip()
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    new_sentence = '<start> ' + new_sentence + ' <end>'
    #print(new_sentence)
    return new_sentence

def reverse_sentence(sentence):
    wordlist=""
    wordlist=sentence.split(" ")
    reversed_wordlist=wordlist[-1::-1]
    reversed_str= ' '.join(reversed_wordlist)
    return reversed_str

def get_dataset_size(path):
    lines = open(path, encoding='UTF-8',errors='ignore').read().strip().split('\n')
    return len(lines)

def create_dataset_word(path, num_examples):
    lines = open(path, encoding='UTF-8',errors='ignore').read().strip().split('\n')
    word_pairs = [[preprocess_sentence_word(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    return word_pairs

def create_dataset_word_qam3classes(path, num_examples):
    lines = open(path, encoding='UTF-8',errors='ignore').read().strip().split('\n')
    #word_pairs = [[preprocess_sentence_word(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    dataset=[]
    for qaline in lines[:num_examples]:
        #print(qaline)
        quest,answr1,match1,answr2,match2,answr3,match3= qaline.split('\t')
        quest_processed= preprocess_sentence_word(quest)
        answr1_processed= preprocess_sentence_word(answr1)
        answr2_processed= preprocess_sentence_word(answr2)
        answr3_processed= preprocess_sentence_word(answr3)
        dataset.append([quest_processed,answr1_processed,match1,answr2_processed,match2,answr3_processed,match3])
    return dataset

def create_dataset_word_qam(path, num_examples):
    lines = open(path, encoding='UTF-8',errors='ignore').read().strip().split('\n')
    #word_pairs = [[preprocess_sentence_word(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    dataset=[]
    for qaline in lines[:num_examples]:
        #print(qaline)
        quest,answr,match= qaline.split('\t')
        quest_processed= preprocess_sentence_word(quest)
        answr_processed= preprocess_sentence_word(answr)
        dataset.append([quest_processed,answr_processed,match])
    return dataset


class EmbeddingIndex():
  def __init__(self, sentence):
    self.sentence = sentence
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()
    
    self.create_index()
    
  def create_index(self):
    for phrase in self.sentence:
      self.vocab.update(phrase.split(' '))
    
    self.vocab = sorted(self.vocab)
    
    self.word2idx['<pad>'] = 0
 
    for index, word in enumerate(self.vocab):
        self.word2idx[word] = index + 1
    
    for word, index in self.word2idx.items():
      self.idx2word[index] = word

      
def max_length(tensor):
    return max(len(t) for t in tensor)

def remove_last_empty_line(path):
    with open(path) as f_input:
        data = f_input.read().rstrip('\n')
    with open(path, 'w') as f_output:    
        f_output.write(data)
        
def remove_empty_lines(path):
    lines = open(path, encoding='UTF-8',errors='ignore').read().strip().split('\n')
    with open(path, 'w') as f:   
        for line in lines:
            if not line.strip(): 
                continue  # skip the empty line
            else:
                 f.write("{}\n".format(line))
        
def get_vocab(path):
    lines = open(path, encoding='UTF-8',errors='ignore').read().strip().split('\n')
    vocab = EmbeddingIndex(word for word in lines)
    return vocab

def load_dataset_word(path, num_examples, input_vocab, output_vocab):
    # creating cleaned input, output pairs
    pairs = create_dataset_word(path, num_examples)

    # Input sentences
    input_tensor = [[input_vocab.word2idx[word] for word in input_sentence.split(' ')] for input_sentence, output_sentence in pairs]

    # Reverse Input sentences (as suggested in sutskever)
    #comment this out for now
    #reversed_input_tensor = [[input_vocab.word2idx[word] for word in reverse_sentence(input_sentence).split(' ')] for input_sentence, output_sentence in pairs]
    
    # Output sentences
    target_tensor = [[output_vocab.word2idx[word] for word in output_sentence.split(' ')] for input_sentence, output_sentence in pairs]
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    #max_length_inp, max_length_reversed_input, max_length_tar = max_length(input_tensor), max_length(reversed_input_tensor), max_length(target_tensor)
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, maxlen=max_length_inp,  padding='post')
    
    #reversed_input_tensor = tf.keras.preprocessing.sequence.pad_sequences(reversed_input_tensor,  maxlen=max_length_reversed_input, padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, maxlen=max_length_tar,  padding='post')
    
    #return input_tensor, reversed_input_tensor, target_tensor, max_length_inp, max_length_reversed_input, max_length_tar     
    return input_tensor, target_tensor, max_length_inp, max_length_tar     

def load_dataset_word_qam3classes(path, num_examples, input_vocab, output_vocab, label_vocab):
    # creating cleaned input, output pairs
    dataset = create_dataset_word_qam3classes(path, num_examples)

    # Input sentences
    input_tensor = [[input_vocab.word2idx[word] for word in input_sentence.split(' ')] for input_sentence, output1, label1 , output2, label2, output3, label3 in dataset]
    
    # Correct answer
    output1_tensor = [[output_vocab.word2idx[word] for word in output1.split(' ')] for input_sentence, output1, label1 , output2, label2, output3, label3 in dataset]
    output1_label_tensor = [[label_vocab.word2idx[label1]] for input_sentence, output1, label1 , output2, label2, output3, label3 in dataset]

    # partially correct answer
    output2_tensor = [[output_vocab.word2idx[word] for word in output2.split(' ')] for input_sentence, output1, label1 , output2, label2, output3, label3 in dataset]
    output2_label_tensor = [[label_vocab.word2idx[label2]] for input_sentence, output1, label1 , output2, label2, output3, label3 in dataset]

    # wrong answer
    output3_tensor = [[output_vocab.word2idx[word] for word in output3.split(' ')] for input_sentence, output1, label1 , output2, label2, output3, label3 in dataset]
    output3_label_tensor = [[label_vocab.word2idx[label3]] for input_sentence, output1, label1 , output2, label2, output3, label3 in dataset]

    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    #max_length_inp, max_length_reversed_input, max_length_tar = max_length(input_tensor), max_length(reversed_input_tensor), max_length(target_tensor)
    max_length_inp, max_length_tar, max_length_label = max_length(input_tensor), max_length(output1_tensor), max_length(output1_label_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, maxlen=max_length_inp,  padding='post')
    
    output1_tensor = tf.keras.preprocessing.sequence.pad_sequences(output1_tensor, maxlen=max_length_tar,  padding='post')
    output2_tensor = tf.keras.preprocessing.sequence.pad_sequences(output2_tensor, maxlen=max_length_tar,  padding='post')
    output3_tensor = tf.keras.preprocessing.sequence.pad_sequences(output3_tensor, maxlen=max_length_tar,  padding='post')
    
    output1_label_tensor = tf.keras.preprocessing.sequence.pad_sequences(output1_label_tensor, maxlen=max_length_label,  padding='post')
    output2_label_tensor = tf.keras.preprocessing.sequence.pad_sequences(output2_label_tensor, maxlen=max_length_label,  padding='post')
    output3_label_tensor = tf.keras.preprocessing.sequence.pad_sequences(output3_label_tensor, maxlen=max_length_label,  padding='post')
    
    #return input_tensor, reversed_input_tensor, target_tensor, max_length_inp, max_length_reversed_input, max_length_tar     
    return input_tensor, output1_tensor, output1_label_tensor, output2_tensor, output2_label_tensor, output3_tensor, output3_label_tensor

def log_epoch_loss(path,epoch, loss):
    now=datetime.today().isoformat()
    with open(path, 'a') as log_file:
        log_file.write("{},{},{}\n".format(now,epoch,loss))
        
#MODEL
def gru(units,dropout, recurrent_dropout,go_backwards=False):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  if tf.test.is_gpu_available():
    return tf.keras.layers.CuDNNGRU(units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform',
                                    go_backwards=go_backwards)
  else:
    return tf.keras.layers.GRU(units, 
                               return_sequences=True, 
                               return_state=True, 
                               dropout=dropout,
                               recurrent_dropout=recurrent_dropout,
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform',
                               go_backwards=go_backwards)
    
def lstm(units,dropout, recurrent_dropout,go_backwards=False):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  #dropout not supported in GPU
  if tf.test.is_gpu_available():
    return tf.keras.layers.CuDNNLSTM(units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform',
                                    go_backwards=go_backwards)
  else:
    return tf.keras.layers.LSTM(units, 
                               return_sequences=True, 
                               return_state=True,
                               dropout=dropout,
                               recurrent_dropout=recurrent_dropout,
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform',
                               go_backwards=go_backwards)
  
class Encoder_BidirectionalGRU(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz, dropout,recurrent_dropout):
        super(Encoder_BidirectionalGRU, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru_fw = gru(self.enc_units,dropout,recurrent_dropout)
        self.gru_bw = gru(self.enc_units,dropout,recurrent_dropout,True)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output_fw, state_fw = self.gru_fw(x, initial_state = hidden)        
        output_bw, state_bw = self.gru_bw(x, initial_state = hidden)  
        output =  tf.concat([output_fw,output_bw], axis=2)
        state =  tf.concat([state_fw,state_bw], axis=1)
        return output, state, x
    
    def initialize_states(self):
        return tf.zeros((self.batch_sz, self.enc_units))
      

class Encoder_BidirectionalLSTM(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz, dropout,recurrent_dropout):
        super(Encoder_BidirectionalLSTM, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm_fw = lstm(self.enc_units,dropout,recurrent_dropout)
        self.lstm_bw = lstm(self.enc_units,dropout,recurrent_dropout,True)
        
    def call(self, x, hidden, cell_state):
        x = self.embedding(x)
        output_fw, state_fw, cell_state_fw = self.lstm_fw(x, initial_state = [hidden, cell_state])        
        output_bw, state_bw, cell_state_bw = self.lstm_bw(x, initial_state = [hidden, cell_state])        
        output =  tf.concat([output_fw,output_bw], axis=2)
        state =  tf.concat([state_fw,state_bw], axis=1)
        state =  tf.concat([cell_state_fw,cell_state_bw], axis=1)
        return output, state, cell_state
   
    def initialize_states(self):
        return tf.zeros((self.batch_sz, self.enc_units))

        
class Decoder_GRU(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, dropout,recurrent_dropout):
        super(Decoder_GRU, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units,dropout,recurrent_dropout)
           
        self.fc = tf.keras.layers.Dense(vocab_size)
               
        # used for attention (Bahdanau style)
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, enc_output, hidden):

        # enc_output shape == (batch_size, max_length, hidden_size)
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, hidden_size)
        #Bahdanau's additive style (Bahdanau attention)        

        #original was tanh
        #score = tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis))
        score = tf.nn.relu6(self.W1(enc_output) + self.W2(hidden_with_time_axis))
        
        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
            
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
            
        #return x, state, attention_weights
        return x, state
 
       
    def initialize_states(self):
        return tf.zeros((self.batch_sz, self.dec_units))
         
class Decoder_LSTM(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, dropout,recurrent_dropout):
        super(Decoder_LSTM, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = lstm(self.dec_units,dropout,recurrent_dropout)
        self.fc = tf.keras.layers.Dense(vocab_size)
               
        # used for attention (Bahdanau style)
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, enc_output, hidden, cell_state):

        # enc_output shape == (batch_size, max_length, hidden_size)
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, hidden_size)
        #Bahdanau's additive style (Bahdanau attention)        

        #original was tanh
        #score = tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis))
        score = tf.nn.relu6(self.W1(enc_output) + self.W2(hidden_with_time_axis))
        
        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the LSTM
        output, state, cell_state = self.lstm(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
            
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
            
        #return x, state, attention_weights
        return x, state, cell_state
           
       
    def initialize_states(self):
        return tf.zeros((self.batch_sz, self.dec_units))

class SimilarityDecoder(tf.keras.Model):
    def __init__(self, vocab_size, units, dropout):
        super(SimilarityDecoder, self).__init__()
        self.fc = tf.keras.layers.Dense(vocab_size)
        
    def call(self, context_vector_question, context_vector_answer):

        context_vector = context_vector_question * context_vector_answer
        
        x=self.fc(context_vector)
       
        return x
        
class SimilarityDecoder_RYANLOWE(tf.keras.Model):
    def __init__(self, vocab_size, units, dropout):
        super(SimilarityDecoder_RYANLOWE, self).__init__()
        
        self.M = tf.keras.layers.Dense(units*2)
        #self.fc = tf.keras.layers.Dense(vocab_size)
        self.fc = tf.keras.layers.Dense(vocab_size,activation='softmax')
        
    def call(self, context_vector_question, context_vector_answer):

        predicted_question = self.M(context_vector_answer)
        dot_product = predicted_question * context_vector_question
        x=self.fc(dot_product)
        return x
      
def generate_samples(input_file, sample_file,lines_count, samples_count):
    import itertools
    import random

    def random_gen(low, high):
        while True:
            yield random.randrange(low, high)

    gen = random_gen(1, lines_count)

    items = set()

    # try to add elem to set until set length is less than 10
    for x in itertools.takewhile(lambda x: len(items) < samples_count+1, gen): 
        items.add(x)
    #print(items)
  
    with open(sample_file, 'w') as f:
        lines = open(input_file, encoding='UTF-8', errors='ignore').read().strip().split('\n')
        line_number=0
        for line in lines:
            line_number  += 1
            if line_number in items:
                f.write("{}\n".format(line))
    
    remove_empty_lines(sample_file)
    remove_last_empty_line(sample_file)
    
def get_samples(path):
    questions=[]
    answers=[]
    lines = open(path, encoding='UTF-8',errors='ignore').read().strip().split('\n')
    lines2= filter(lambda x: not x.isspace(), lines)
    for line in lines2:
        questions.append(line.split('\t')[0])
        answers.append(line.split('\t')[1])
    return questions, answers

def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)

def get_test_questions(path, start_row, num_examples):
    lines = open(path, encoding='UTF-8',errors='ignore').read().strip().split('\n')
    end_row=start_row+num_examples
    word_pairs = [l.split('\t') for l in lines[start_row:end_row]]
    return word_pairs

def greedy_search_prediction(question, rnntype, encoder, decoder, input_vocab, output_vocab, input_max_len, output_max_len):
    #attention_plot = np.zeros((max_length_targ, max_length_inp))
   
    inputs = [input_vocab.word2idx[i] for i in question.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=input_max_len, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = encoder.initialize_states()
    if rnntype == 'LSTM':
        enc_cell = encoder.initialize_states()
        dec_cell = decoder.initialize_states()
        enc_out, enc_hidden, enc_cell = encoder(inputs, hidden, enc_cell)
    else:
        enc_out, enc_hidden = encoder(inputs, hidden)
        
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([output_vocab.word2idx['<start>']], 0)

    #print(output_vocab.idx2word)
    score_highest=[]
    #score_lowest=[]
    #score_mean=[]
    #score_total=[]
    
    for t in range(output_max_len):
      
        if rnntype == 'LSTM':
          #predictions, dec_hidden, dec_cell, attention_weights = decoder(dec_input, enc_out, dec_hidden, dec_cell )
          predictions, dec_hidden, dec_cell = decoder(dec_input, enc_out, dec_hidden, dec_cell )
        else:
          #predictions, dec_hidden, attention_weights = decoder(dec_input, enc_out, dec_hidden)
          predictions, dec_hidden = decoder(dec_input, enc_out, dec_hidden)
        #print("Predictions is: {}".format(predictions))
    
        #predicted_id = tf.multinomial(predictions, num_samples=1)[0][0].numpy()
        predicted_id = tf.argmax(predictions[0]).numpy()
        
        highest = (tf.reduce_max(predictions[0])).numpy()
        #lowest  = (tf.reduce_min(predictions[0])).numpy()
        #mean    = (tf.reduce_mean(predictions[0])).numpy()
        #total   = (tf.reduce_sum(predictions[0])).numpy()
      
        score_highest.append(highest)
        #score_lowest.append(lowest)        
        #score_mean.append(mean)        
        #score_total.append(total) 
        
        result += output_vocab.idx2word[predicted_id] + ' '
        
        if output_vocab.idx2word[predicted_id] == '<end>':
            #return result, sentence, attention_plot
            high_score = sum(score_highest)/(t+1)
            #low_score = sum(score_lowest)/(t+1)    
            #mean_score = sum(score_mean)/(t+1)
            #total_score = sum(score_total)/(t+1)    
            #bleu_score = calculate_bleu(clean_string(reference_answer),clean_string(result))
            return result, high_score
        
            # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)
    
    #return result, sentence, attention_plot
    high_score = sum(score_highest)/(t+1)
    #low_score = sum(score_lowest)/(t+1)    
    #mean_score = sum(score_mean)/(t+1)    
    #total_score = sum(score_total)/(t+1)   
    #bleu_score = calculate_bleu(reference_answer,result)    
    
    return result, high_score

def get_k_top_predictions(predictions,avg_sent_prob ,predicted_sentence, dec_input,k=1):
    #the reason sentence_prob,predicted_sentence, dec_input is passed is that i can use it back unchanged
    #there is no processing done for these values bcos i need it unchnaged in the calling function
    k_top_predictions = tf.nn.top_k(predictions[0],k)
    predictedids = k_top_predictions[1].numpy()
    probabilities = k_top_predictions[0].numpy()
    #create is, prob pair of list
    id_prob_pair=[]
    for idx in range(k):
        id_prob_pair.append([predictedids[idx],probabilities[idx],avg_sent_prob ,predicted_sentence, dec_input])
    return id_prob_pair
    

def beam_search_prediction(question, rnntype, encoder, decoder, input_vocab, output_vocab, input_max_len, output_max_len,k=1):
    #attention_plot = np.zeros((max_length_targ, max_length_inp))
   
    inputs = [input_vocab.word2idx[i] for i in question.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=input_max_len, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    predicted_sentence = ''
   
    hidden = encoder.initialize_states()
    if rnntype == 'LSTM':
        enc_cell = encoder.initialize_states()
        dec_cell = decoder.initialize_states()
        enc_out, enc_hidden, enc_cell = encoder(inputs, hidden, enc_cell)
    else:
        enc_out, enc_hidden , embedding = encoder(inputs, hidden)
    
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([output_vocab.word2idx['<start>']], 0)
    #idea from https://github.com/mmehdig/lm_beam_search/blob/master/beam_search.py
    #K_beam[prob,result,dec_input,dec_hidden]
    total_sent_prob = 0 #average sentence probability, this is gettting the average (add all probs and divide by t)
    k_beam = [(total_sent_prob , predicted_sentence,dec_input,dec_hidden)]
    
    
    for t in range(output_max_len):
        all_k_beams = []
        for total_sent_prob , predicted_sentence, dec_input, dec_hidden in k_beam:

            predictions, dec_hidden = decoder(dec_input, enc_out, dec_hidden)
            possible_k = get_k_top_predictions(predictions,total_sent_prob, predicted_sentence, dec_input,k)
            
            for predicted_id, word_prob,total_sent_prob,predicted_sentence, dec_input in possible_k:
                    
                if output_vocab.idx2word[predicted_id] == '<end>':
                    #predicted_sentence += output_vocab.idx2word[predicted_id] + ' '
                    predicted_sentence += '<end>' 
                    total_sent_prob += word_prob
                    dec_input = tf.expand_dims([predicted_id], 0)
                    all_k_beams +=[(total_sent_prob, predicted_sentence,dec_input,dec_hidden)]
                else:
                    predicted_sentence += output_vocab.idx2word[predicted_id] + ' '
                    total_sent_prob += word_prob
                    dec_input = tf.expand_dims([predicted_id], 0)
                    all_k_beams +=[(total_sent_prob, predicted_sentence,dec_input,dec_hidden)]
                
            k_beam = sorted(all_k_beams,reverse=True)[:k]
            #k_beam = sorted(all_k_beams)[:k]
            predicted_sent_tmp = k_beam[0][1]
            if '<end>' in predicted_sent_tmp:
                avg_sent_prob = (k_beam[0][0])/(t+1)
                return predicted_sent_tmp, avg_sent_prob

    #get top 1 for now
    t_beam = sorted(k_beam,reverse=True)[:1]
    #t_beam = sorted(k_beam)[:1]
    #sentence_prob_pair=[]
    #for sentence_prob, total_sent_prob, predicted_sentence, _ , _ in t_beam:
    #    avg_sent_prob = total_sent_prob/t
    #    sentence_prob_pair.append([sentence_prob, avg_sent_prob, predicted_sentence])

    predicted_sent = t_beam[0][1]
    #if t==0:
    #    avg_sent_prob = 0
    #else:
    #need to get average
    avg_sent_prob = (t_beam[0][0])/(t+1)
    return predicted_sent, avg_sent_prob

def remove_everything_after_end_token(sentence):
    searchStr = '<end>'
    return sentence[:sentence.find(searchStr)]

print("Helper Functions Done")

Helper Functions Done


In [5]:
#@title
#Variables' definitions

param = Settings(settings_file_path,dataset_folder)
corpus_path = param.training_set_path#input from user
param = Settings(settings_file_path,dataset_folder)
training_set_path = param.training_set_path
validation_set_path = param.validation_set_path
input_vocab_path=param.input_vocab_path
output_vocab_path=param.output_vocab_path
label_vocab_path=param.label_vocab_path_qam3
checkpoint_path = param.checkpoint_path
epochs = int(param.epochs)
batch_size = int(param.batch_size)
embedding_dim = int(param.embedding_dim)
units  = int(param.units)
training_loss_path = param.training_loss_path
validation_loss_path = param.validation_loss_path
encoder_type=param.encoder_type
embedding_type=param.embedding_type
rnn_type=param.rnn_type
training_size = int(param.training_size)
validation_size = int(param.validation_size)
dropout=float(param.dropout)
recurrent_dropout=float(param.recurrent_dropout)
internal_feedback=param.internal_feedback
test_questions_path=param.test_set_path
predictions_path=param.predictions_path
input_max_len = int(param.input_max_len)
output_max_len = int(param.output_max_len)
beam_size=int(param.beam_size)
rnntype=rnn_type.upper()
input_vocab = get_vocab(input_vocab_path)
output_vocab = get_vocab(output_vocab_path)
label_vocab = get_vocab(label_vocab_path)
input_vocab_size = len(input_vocab.word2idx)
output_vocab_size = len(output_vocab.word2idx)
label_vocab_size=len(label_vocab.word2idx)

rnntype=rnn_type.upper()

print('Model Details')
print('Training size is {}'.format(training_size))
print("Vocab size is {}:{}".format(input_vocab_size,output_vocab_size))
print('Epoch count is {}'.format(epochs))
print('Embedding size is {}'.format(embedding_dim))
print('Hidden units is {}'.format(units))
print('Batch size is {}'.format(batch_size))

Model Details
Training size is 30000
Vocab size is 12270:11471
Epoch count is 20
Embedding size is 512
Hidden units is 512
Batch size is 32


In [6]:
#@title
"""
#Testing - standard beam search
batch_size=1   
if rnntype == 'GRU':
    qam3_question_encoder = Encoder_BidirectionalGRU(input_vocab_size, embedding_dim, units, batch_size, dropout, recurrent_dropout)
    qam3_answer_decoder = Decoder_GRU(output_vocab_size, embedding_dim, units*2, batch_size, dropout, recurrent_dropout)
    qam3_answer_encoder = Encoder_BidirectionalGRU(output_vocab_size, embedding_dim, units, batch_size, dropout, recurrent_dropout)
    qam3_label_decoder = SimilarityDecoder_RYANLOWE(label_vocab_size, units, dropout)
    
optimizer = tf.train.AdamOptimizer() #tf v1
#optimizer = tf.optimizers.Adam()  #tf v2

#prepare checkpoint details
checkpoint_metafilename='checkpoint'
checkpoint_filepath = os.path.join(checkpoint_path,checkpoint_metafilename)
#checkpoint_prefix = os.path.join(checkpoint_path, "ckpt")
if os.path.exists(checkpoint_filepath):
  print("Reloading existing checkpoint file")
  checkpoint = tf.train.Checkpoint(optimizer=optimizer, qam3_question_encoder=qam3_question_encoder, qam3_answer_decoder=qam3_answer_decoder, qam3_answer_encoder = qam3_answer_encoder, qam3_label_decoder=qam3_label_decoder)
  checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))
  print("Existing checkpoint file loaded")
else:
  print("Checkpoint file is missing")

start_row=0
test_size=1000
QA_Pairs = get_test_questions (test_questions_path,start_row, test_size)
print('Questions path {}'.format(test_questions_path))
qa_pair=[]

filename, file_extension = os.path.splitext(predictions_path)
#beam_search_filename = filename + 'BS_' +  embedding_type + '_'+ encoder_type +'_'+ rnn_type +'_'+ str(dropout) +'_'+ internal_feedback + '.txt'
beam_search_filename = filename + '_standard_bs_QAM3.txt'
print('File path: {}'.format(beam_search_filename))
#predictions_path_gs = filename + '_greedy.txt'
#predictions_path_bs = filename + '_bs_' + str(beam_size) + '.csv'
#just focus on beam search  

qa_pair_bs=[]

#beam search
for question, answer in QA_Pairs:
    if embedding_type == 'word':
        #takes care for char and mix
        cleaned_input = preprocess_user_input_word(question,input_vocab)
    else:
        cleaned_input = preprocess_sentence_char(question)
    
    if len(cleaned_input.strip()):
        reply,  score = beam_search_prediction(cleaned_input, rnntype, qam3_question_encoder, qam3_answer_decoder, input_vocab, output_vocab, input_max_len, output_max_len,beam_size)    
        
        if embedding_type == 'char':
            reply = preprocess_output_char(reply)
 
        reply = re.sub('<start>', '', reply)
        #reply = re.sub('<end>', '', reply)
        
        idx = reply.find('<end>')
        if idx > 0:
            reply = reply[:idx]
        else:
            reply = re.sub('<end>', '', reply)
        
        reply = re.sub('<pad>', '', reply)
        #print(reply)
        qa_pair_bs.append(question.strip()+'\t'+  answer.strip() + '\t' + reply.strip() )
    else:
        reply = 'Zero words/characters in vocabulary'
        score=-1
        qa_pair_bs.append(question.strip()+'\t'+  answer.strip() + '\t' + reply.strip() )
        
with open(beam_search_filename, 'w') as f:
    for line in qa_pair_bs:
        f.write("{}\n".format(line))

print('Standard beam search done. File created at {}'.format(beam_search_filename))
"""

'\n#Testing - standard beam search\nbatch_size=1   \nif rnntype == \'GRU\':\n    qam3_question_encoder = Encoder_BidirectionalGRU(input_vocab_size, embedding_dim, units, batch_size, dropout, recurrent_dropout)\n    qam3_answer_decoder = Decoder_GRU(output_vocab_size, embedding_dim, units*2, batch_size, dropout, recurrent_dropout)\n    qam3_answer_encoder = Encoder_BidirectionalGRU(output_vocab_size, embedding_dim, units, batch_size, dropout, recurrent_dropout)\n    qam3_label_decoder = SimilarityDecoder_RYANLOWE(label_vocab_size, units, dropout)\n    \noptimizer = tf.train.AdamOptimizer() #tf v1\n#optimizer = tf.optimizers.Adam()  #tf v2\n\n#prepare checkpoint details\ncheckpoint_metafilename=\'checkpoint\'\ncheckpoint_filepath = os.path.join(checkpoint_path,checkpoint_metafilename)\n#checkpoint_prefix = os.path.join(checkpoint_path, "ckpt")\nif os.path.exists(checkpoint_filepath):\n  print("Reloading existing checkpoint file")\n  checkpoint = tf.train.Checkpoint(optimizer=optimizer, q

In [7]:
#Testing
"""
def QAM3_beam_search_prediction(question, rnntype, encoder, decoder, question_encoder, answer_encoder, label_decoder, input_vocab, output_vocab, label_vocab, input_max_len, output_max_len,k=1):
    #attention_plot = np.zeros((max_length_targ, max_length_inp))
   
    inputs = [input_vocab.word2idx[i] for i in question.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=input_max_len, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    predicted_sentence = ''
   
    hidden = encoder.initialize_states()
    enc_out, enc_hidden, _ = encoder(inputs, hidden)
    
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([output_vocab.word2idx['<start>']], 0)
    total_sent_prob = 0 #average sentence probability, this is gettting the average (add all probs and divide by t)
    k_beam = [(total_sent_prob , predicted_sentence,dec_input,dec_hidden)]
    
    output_beam =[]
    
    for t in range(output_max_len):
        all_k_beams = []
        
        for total_sent_prob , predicted_sentence, dec_input, dec_hidden in k_beam:
            
            predictions, dec_hidden = decoder(dec_input, enc_out, dec_hidden)
            possible_k = get_k_top_predictions(predictions,total_sent_prob, predicted_sentence, dec_input,k)
            
            for predicted_id, word_prob,total_sent_prob,predicted_sentence, dec_input in possible_k:
                    
                if output_vocab.idx2word[predicted_id] == '<end>':
                    predicted_sentence += '<end>' 
                else:
                    predicted_sentence += output_vocab.idx2word[predicted_id] + ' '

                total_sent_prob += word_prob
                dec_input = tf.expand_dims([predicted_id], 0)
                all_k_beams +=[(total_sent_prob, predicted_sentence,dec_input,dec_hidden)]
                
            k_beam = sorted(all_k_beams,reverse=True)[:k]
    
    for i in range(len(k_beam)):
        output_beam.append([k_beam[i][0], remove_everything_after_end_token(k_beam[i][1])])
    
    updated_output_beam=[]
    for replies in output_beam:
        score = replies[0]
        reply = preprocess_user_input_word(replies[1] ,output_vocab)
        question_inputs = [input_vocab.word2idx[i] for i in question.split(' ')]
        question_inputs = tf.keras.preprocessing.sequence.pad_sequences([question_inputs], maxlen=input_max_len, padding='post')
        question_inputs = tf.convert_to_tensor(question_inputs )

        reply_outputs = [output_vocab.word2idx[i] for i in reply.split(' ')]
        reply_outputs = tf.keras.preprocessing.sequence.pad_sequences([reply_outputs], maxlen=output_max_len, padding='post')
        reply_outputs = tf.convert_to_tensor(reply_outputs)

        question_hidden = question_encoder.initialize_states()
        answer_hidden = answer_encoder.initialize_states()

        question_enc_output, question_enc_hidden, _ = question_encoder(question_inputs, question_hidden)
        answer_enc_output, answer_enc_hidden , _ = answer_encoder(reply_outputs, answer_hidden)
        predictions = label_decoder(question_enc_hidden , answer_enc_hidden )
        predicted_id = tf.argmax(predictions[0]).numpy()
        if label_vocab.idx2word[predicted_id] != '<pad>':
          matching_percentage = int(label_vocab.idx2word[predicted_id])
        else:
          matching_percentage = 0

        new_score = (score + score*matching_percentage/100)/2
        updated_output_beam.append([new_score,reply])
        sorted_updated_output_beam = sorted(updated_output_beam,reverse=True)
    
    predicted_sent = sorted_updated_output_beam [0][1]
    avg_sent_prob = (sorted_updated_output_beam [0][0])/(t+1)
    
    return predicted_sent, avg_sent_prob
print('Enhanced beam search function definition done')
#Testing using Enhanced Beam Search - QAM3
batch_size=1   

qam3_question_encoder = Encoder_BidirectionalGRU(input_vocab_size, embedding_dim, units, batch_size, dropout, recurrent_dropout)
qam3_answer_decoder = Decoder_GRU(output_vocab_size, embedding_dim, units*2, batch_size, dropout, recurrent_dropout)
qam3_answer_encoder = Encoder_BidirectionalGRU(output_vocab_size, embedding_dim, units, batch_size, dropout, recurrent_dropout)
qam3_label_decoder = SimilarityDecoder_RYANLOWE(label_vocab_size, units, dropout)
    
optimizer = tf.train.AdamOptimizer() #tf v1
#optimizer = tf.optimizers.Adam()  #tf v2

#prepare checkpoint details
checkpoint_metafilename='checkpoint'
checkpoint_filepath = os.path.join(checkpoint_path,checkpoint_metafilename)
checkpoint_prefix = os.path.join(checkpoint_path, "ckpt")

if os.path.exists(checkpoint_filepath):
  print("Reloading existing checkpoint file")
  checkpoint = tf.train.Checkpoint(optimizer=optimizer, qam3_question_encoder=qam3_question_encoder, qam3_answer_decoder=qam3_answer_decoder, qam3_answer_encoder = qam3_answer_encoder, qam3_label_decoder=qam3_label_decoder)
  checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))
  print("Existing checkpoint file loaded")
else:
  print("Checkpoint file is missing")

start_row=0
test_size=1000
QA_Pairs = get_test_questions (test_questions_path,start_row, test_size)
print('Questions path {}'.format(test_questions_path))
qa_pair=[]

filename, file_extension = os.path.splitext(predictions_path)
#beam_search_filename = filename + '_qam3_bs_QAM3_' + str(start_row) + 'To' + str(start_row+test_size) +'.txt'
beam_search_filename = filename + '_qam3_bs_QAM3_.txt'
print('File path: {}'.format(beam_search_filename))

#qa_pair_bs=[]

f = open(beam_search_filename, 'w')

#beam search
for question, answer in QA_Pairs:

    #print(question)

    if embedding_type == 'word':
        #takes care for char and mix
        cleaned_input = preprocess_user_input_word(question,input_vocab)
    else:
        cleaned_input = preprocess_sentence_char(question)
    
    if len(cleaned_input.strip()):
        
        reply, score = QAM3_beam_search_prediction(cleaned_input, rnntype, qam3_question_encoder, qam3_answer_decoder,  qam3_question_encoder, qam3_answer_encoder, qam3_label_decoder, input_vocab, output_vocab, label_vocab, input_max_len, output_max_len,beam_size)    
        
        if embedding_type == 'char':
            reply = preprocess_output_char(reply)
 
        reply = re.sub('<start>', '', reply)
        #reply = re.sub('<end>', '', reply)
        
        idx = reply.find('<end>')
        if idx > 0:
            reply = reply[:idx]
        else:
            reply = re.sub('<end>', '', reply)
        
        reply = re.sub('<pad>', '', reply)
        #print(reply)
        #qa_pair_bs.append(question.strip()+'\t'+  answer.strip() + '\t' + reply.strip() )
        line=question.strip()+'\t'+  answer.strip() + '\t' + reply.strip() 
        f.write("{}\n".format(line))
    else:
        reply = 'Zero words/characters in vocabulary'
        score=-1
        #qa_pair_bs.append(question.strip()+'\t'+  answer.strip() + '\t' + reply.strip() )
        line=question.strip()+'\t'+  answer.strip() + '\t' + reply.strip() 
        f.write("{}\n".format(line))
        
#with open(beam_search_filename, 'w') as f:
#    for line in qa_pair_bs:
#        f.write("{}\n".format(line))
f.close()
print('QAM3 Beam search done. File created at {}'.format(beam_search_filename))
"""

'\ndef QAM3_beam_search_prediction(question, rnntype, encoder, decoder, question_encoder, answer_encoder, label_decoder, input_vocab, output_vocab, label_vocab, input_max_len, output_max_len,k=1):\n    #attention_plot = np.zeros((max_length_targ, max_length_inp))\n   \n    inputs = [input_vocab.word2idx[i] for i in question.split(\' \')]\n    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=input_max_len, padding=\'post\')\n    inputs = tf.convert_to_tensor(inputs)\n    \n    predicted_sentence = \'\'\n   \n    hidden = encoder.initialize_states()\n    enc_out, enc_hidden, _ = encoder(inputs, hidden)\n    \n    dec_hidden = enc_hidden\n    dec_input = tf.expand_dims([output_vocab.word2idx[\'<start>\']], 0)\n    total_sent_prob = 0 #average sentence probability, this is gettting the average (add all probs and divide by t)\n    k_beam = [(total_sent_prob , predicted_sentence,dec_input,dec_hidden)]\n    \n    output_beam =[]\n    \n    for t in range(output_max_len)

In [0]:
#Prediction Chatbot

def QAM3_beam_search_prediction(question, rnntype, encoder, decoder, question_encoder, answer_encoder, label_decoder, input_vocab, output_vocab, label_vocab, input_max_len, output_max_len,k=1):
   
    inputs = [input_vocab.word2idx[i] for i in question.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=input_max_len, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    predicted_sentence = ''
   
    hidden = encoder.initialize_states()
    enc_out, enc_hidden, _ = encoder(inputs, hidden)
    
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([output_vocab.word2idx['<start>']], 0)
    total_sent_prob = 0 #average sentence probability, this is gettting the average (add all probs and divide by t)
    k_beam = [(total_sent_prob , predicted_sentence,dec_input,dec_hidden)]
    
    output_beam =[]
    
    for t in range(output_max_len):
        all_k_beams = []
        
        for total_sent_prob , predicted_sentence, dec_input, dec_hidden in k_beam:
            
            predictions, dec_hidden = decoder(dec_input, enc_out, dec_hidden)
            possible_k = get_k_top_predictions(predictions,total_sent_prob, predicted_sentence, dec_input,k)
            
            for predicted_id, word_prob,total_sent_prob,predicted_sentence, dec_input in possible_k:
                    
                if output_vocab.idx2word[predicted_id] == '<end>':
                    predicted_sentence += '<end>' 
                else:
                    predicted_sentence += output_vocab.idx2word[predicted_id] + ' '

                total_sent_prob += word_prob
                dec_input = tf.expand_dims([predicted_id], 0)
                all_k_beams +=[(total_sent_prob, predicted_sentence,dec_input,dec_hidden)]
                
            k_beam = sorted(all_k_beams,reverse=True)[:k]
    
    for i in range(len(k_beam)):
        output_beam.append([k_beam[i][0], remove_everything_after_end_token(k_beam[i][1])])
    
    updated_output_beam=[]
    for replies in output_beam:
        score = replies[0]
        reply = preprocess_user_input_word(replies[1] ,output_vocab)
        question_inputs = [input_vocab.word2idx[i] for i in question.split(' ')]
        question_inputs = tf.keras.preprocessing.sequence.pad_sequences([question_inputs], maxlen=input_max_len, padding='post')
        question_inputs = tf.convert_to_tensor(question_inputs )

        reply_outputs = [output_vocab.word2idx[i] for i in reply.split(' ')]
        reply_outputs = tf.keras.preprocessing.sequence.pad_sequences([reply_outputs], maxlen=output_max_len, padding='post')
        reply_outputs = tf.convert_to_tensor(reply_outputs)

        question_hidden = question_encoder.initialize_states()
        answer_hidden = answer_encoder.initialize_states()

        question_enc_output, question_enc_hidden, _ = question_encoder(question_inputs, question_hidden)
        answer_enc_output, answer_enc_hidden , _ = answer_encoder(reply_outputs, answer_hidden)
        predictions = label_decoder(question_enc_hidden , answer_enc_hidden )
        predicted_id = tf.argmax(predictions[0]).numpy()
        if label_vocab.idx2word[predicted_id] != '<pad>':
          matching_percentage = int(label_vocab.idx2word[predicted_id])
        else:
          matching_percentage = 0

        new_score = (score + score*matching_percentage/100)/2
        updated_output_beam.append([new_score,reply])
        sorted_updated_output_beam = sorted(updated_output_beam,reverse=True)
    
    predicted_sent = sorted_updated_output_beam [0][1]
    avg_sent_prob = (sorted_updated_output_beam [0][0])/(t+1)
    
    return predicted_sent, avg_sent_prob

#print('Enhanced beam search function definition done')
#Testing using Enhanced Beam Search - QAM3
batch_size=1   

qam3_question_encoder = Encoder_BidirectionalGRU(input_vocab_size, embedding_dim, units, batch_size, dropout, recurrent_dropout)
qam3_answer_decoder = Decoder_GRU(output_vocab_size, embedding_dim, units*2, batch_size, dropout, recurrent_dropout)
qam3_answer_encoder = Encoder_BidirectionalGRU(output_vocab_size, embedding_dim, units, batch_size, dropout, recurrent_dropout)
qam3_label_decoder = SimilarityDecoder_RYANLOWE(label_vocab_size, units, dropout)
    
optimizer = tf.train.AdamOptimizer() #tf v1
#optimizer = tf.optimizers.Adam()  #tf v2

#prepare checkpoint details
checkpoint_metafilename='checkpoint'
checkpoint_filepath = os.path.join(checkpoint_path,checkpoint_metafilename)
checkpoint_prefix = os.path.join(checkpoint_path, "ckpt")

if os.path.exists(checkpoint_filepath):
  print("Reloading existing checkpoint file")
  checkpoint = tf.train.Checkpoint(optimizer=optimizer, qam3_question_encoder=qam3_question_encoder, qam3_answer_decoder=qam3_answer_decoder, qam3_answer_encoder = qam3_answer_encoder, qam3_label_decoder=qam3_label_decoder)
  checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))
  print("Existing checkpoint file loaded")
else:
  print("Checkpoint file is missing")


print("ChatBot: Hello there !")
while (True):
    question = input("You: ")
    if question.lower() == 'goodbye':
        print("ChatBot: Bye bye, see you soon!")
        break
    if embedding_type == 'word':
        #takes care for char and mix
        cleaned_input = preprocess_user_input_word(question,input_vocab)
    else:
        cleaned_input = preprocess_sentence_char(question)
   
    if len(cleaned_input.strip()):
        
        reply, score = QAM3_beam_search_prediction(cleaned_input, rnntype, qam3_question_encoder, qam3_answer_decoder,  qam3_question_encoder, qam3_answer_encoder, qam3_label_decoder, input_vocab, output_vocab, label_vocab, input_max_len, output_max_len,beam_size)    
 
        reply = re.sub('<start>', '', reply)
        idx = reply.find('<end>')
        if idx > 0:
            reply = reply[:idx]
        else:
            reply = re.sub('<end>', '', reply)
        reply = re.sub('<pad>', '', reply)
        print('Chatbot: ' + reply)
    else:
        reply = 'Sorry, I could not under stand anything you said.'
        score=-1
        print('Chatbot: ' + reply)


Reloading existing checkpoint file
Existing checkpoint file loaded
ChatBot: Hello there !
You: Hi, how are you ?
Chatbot:  how are you staying away 
You: anybody know what today for a great day is ?
Chatbot:  to take your dogs on a walk with your girlfriend 
You: wait are you coming home too ?
Chatbot:  from the first dance 
You: wow u ! so talented 
Chatbot:  thank you 
You: why ? tech companies in need of diverse teams and people in need of can help each other .
Chatbot:  cool ! lmk if you could use a freelance professor -- filled out the volunteer 
You: takes one to know one
Chatbot:  yes does but you def win 
You: happy birthday
Chatbot:  awesome thank you so much i appreciate it 
You: should i trade spencer ware for jordan matthews standard scoring . season long
Chatbot:  ok with that ! 
You: how are you doing ?
Chatbot:  lol im at the bus stop 
You: where are you now ?
Chatbot:  new york university 
You: what are you doing there ?
Chatbot:  an installation at 10 pm est ? 
